In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Pipelines: Evaluating batch prediction results from AutoML Tabular regression model

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/automl_tabular_regression_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/automl_tabular_regression_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_evaluation/automl_tabular_regression_model_evaluation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This notebook demonstrates how to use Vertex AI regression model evaluation component to evaluate an AutoML Tabular regression model. Model evaluation helps you determine your model performance based on the evaluation metrics and improve the model if necessary. 

Learn more about [Vertex AI Model Evaluation](https://cloud.google.com/vertex-ai/docs/evaluation/introduction). Learn more about [Regression for tabular data](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/overview).

### Objective

In this tutorial, you learn how to evaluate a Vertex AI model resource through a Vertex AI pipeline job using `google_cloud_pipeline_components`:

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Datasets (Tabular)
- Vertex AI Training (AutoML Tabular Training)
- Vertex AI Batch predictions
- Vertex AI Pipelines
- Vertex AI Model Registry


The steps performed include:

- Create a Vertex AI Dataset
- Configure a `AutoMLTabularTrainingJob`
- Run the `AutoMLTabularTrainingJob` which returns a model
- Import a pre-trained `AutoML model resource` into the pipeline
- Run a `batch prediction` job in the pipeline
- Evaluate the AutoML model using the `regression evaluation component`
- Import the Regression Metrics to the AutoML model resource

### Dataset

The dataset being used in this notebook is a part of the PetFinder Dataset, available [here](https://www.kaggle.com/c/petfinder-adoption-prediction) on Kaggle. The current dataset is only a part of the original dataset considered for the problem of predicting age of the pet. It consists of the following fields:

- `Type`: Type of animal (1 = Dog, 2 = Cat)
- `Age`: Age of pet when listed, in months
- `Breed1`: Primary breed of pet
- `Gender`: Gender of pet
- `Color1`: Color 1 of pet 
- `Color2`: Color 2 of pet
- `MaturitySize`: Size at maturity (1 = Small, 2 = Medium, 3 = Large, 4 = Extra Large, 0 = Not Specified)
- `FurLength`: Fur length (1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified)
- `Vaccinated`: Pet has been vaccinated (1 = Yes, 2 = No, 3 = Not Sure)
- `Sterilized`: Pet has been spayed / neutered (1 = Yes, 2 = No, 3 = Not Sure)
- `Health`: Health Condition (1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 = Not Specified)
- `Fee`: Adoption fee (0 = Free)
- `PhotoAmt`: Total uploaded photos for this pet
- `Adopted`: Whether or not the pet was adopted (Yes/No).

**Note**: This dataset is moved to a public Cloud Storage bucket and is accessed from there in this notebook.

### Costs 
This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook. 


In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-pipeline-components==1.0.26 \
                                 matplotlib 

### Colab only: Uncomment the following cell to restart the kernel

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"
DATA_REGION = "US"

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

#### Service Account

You use a service account to create Vertex AI Pipeline jobs. If you do not want to use your project's Compute Engine service account, set `SERVICE_ACCOUNT` to another service account ID.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### Set service account access for Vertex AI Pipelines

Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step. You only need to run this step once per service account.

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### Import libraries

In [ ]:
import json

import google.cloud.aiplatform as aiplatform
import matplotlib.pyplot as plt
from google.cloud import aiplatform_v1

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## Create Vertex AI Dataset

Create a managed tabular dataset resource in Vertex AI using the dataset source.

In [ ]:
DATA_SOURCE = "gs://cloud-samples-data/ai-platform-unified/datasets/tabular/petfinder-tabular-classification.csv"

In [ ]:
# Create the Vertex AI Dataset resource
dataset = aiplatform.TabularDataset.create(
    display_name="petfinder-tabular-dataset",
    gcs_source=DATA_SOURCE,
)

print("Resource name:", dataset.resource_name)

## Train AutoML model

Train a simple regression model using the created dataset using `Age` as the target column. 

**Set a display name and create the `AutoMLTabularTrainingJob` with appropriate data types specified for column transformations.**

In [ ]:
TRAINING_JOB_DISPLAY_NAME = "[your-train-job-display-name]"  # @param {type:"string"}

In [ ]:
# If no display name is specified, use the default one
if (
    TRAINING_JOB_DISPLAY_NAME == ""
    or TRAINING_JOB_DISPLAY_NAME is None
    or TRAINING_JOB_DISPLAY_NAME == "[your-train-job-display-name]"
):
    TRAINING_JOB_DISPLAY_NAME = "train-pet-agefinder-automl"

### Define AutoML Tabular training job

An AutoML training job is created with the `AutoMLTabularTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `optimization_prediction_type`: The type of prediction the AutoML Model is to produce. Ex: regression, classification.
- `column_transformations`: Transformations to apply to the input columns (i.e. columns other than the targetColumn). Each transformation may produce multiple result values from the column's value, and all are used for training. 
- `optimization_objective`: The optimization objective to minimize or maximize.
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

Learn about [AutoMLTabularTrainingJob](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.AutoMLTabularTrainingJob)    

In [ ]:
train_job = aiplatform.AutoMLTabularTrainingJob(
    display_name=TRAINING_JOB_DISPLAY_NAME,
    optimization_prediction_type="regression",
    column_specs={
        "Type": "categorical",
        "Breed1": "categorical",
        "Gender": "categorical",
        "Color1": "categorical",
        "Color2": "categorical",
        "MaturitySize": "categorical",
        "FurLength": "categorical",
        "Vaccinated": "categorical",
        "Sterilized": "categorical",
        "Health": "categorical",
        "Fee": "numeric",
        "PhotoAmt": "numeric",
        "Adopted": "categorical",
    },
    optimization_objective="minimize-rmse",
)

print(train_job)

#### Set the display name for the model.

In [ ]:
MODEL_DISPLAY_NAME = "[your-model-display-name]"  # @param {type:"string"}

In [ ]:
# If no name is specified, use the default name
if (
    MODEL_DISPLAY_NAME == ""
    or MODEL_DISPLAY_NAME is None
    or MODEL_DISPLAY_NAME == "[your-model-display-name]"
):
    MODEL_DISPLAY_NAME = "pet-agefinder-prediction-model"

#### Run the training job

Next, you start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `target_column`: The name of the column, whose values the model is to predict.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `validation_fraction_split`: The percentage of the dataset to use for validation.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `model_display_name`: The human readable name for the trained model.
- `budget_milli_node_hours`: The train budget of creating this Model, expressed in milli node hours i.e. 1,000 value in this field means 1 node hour. 

The training job takes roughly 3 hours to finish.

In [ ]:
# Run the training job
model = train_job.run(
    dataset=dataset,
    target_column="Age",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    model_display_name=MODEL_DISPLAY_NAME,
    budget_milli_node_hours=1000,
)

## List model evaluations from training

After the training job is finished, get the model evaluations and print them.

In [ ]:
# Get evaluations
model_evaluations = model.list_model_evaluations()

model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

In [ ]:
# Print the evaluation metrics
for evaluation in model_evaluations:
    evaluation = evaluation.to_dict()
    print("Model's evaluation metrics from Training:\n")
    metrics = evaluation["metrics"]
    for metric in metrics.keys():
        print(f"metric: {metric}, value: {metrics[metric]}\n")

## Run a pipeline for model evaluation

Now, you run a Vertex AI BatchPrediction job and generate evaluations and feature-attributions on its results by creating a Vertex AI pipeline using `evaluate` function. Learn more about [evaluate function](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/models.py#L5127).

### Define parameters to run the evaluate function

Specify the required parameters to run `evaluate` function. 

The following is the instruction of `evaluate` function paramters:

- `prediction_type`: The problem type being addressed by this evaluation run. 'classification' and 'regression' are the currently supported problem types.
- `target_field_name`: Name of the column to be used as the target for regression.
- `gcs_source_uris`: List of the Cloud Storage bucket uris of input instances for batch prediction.
- `generate_feature_attributions`: Optional. Whether the model evaluation job should generate feature attributions. Defaults to False if not specified.

**The pipeline takes about 2 hours to complete.**

In [ ]:
job = model.evaluate(
    prediction_type="regression",
    target_field_name="Age",
    gcs_source_uris=[DATA_SOURCE],
    generate_feature_attributions=True,
)

print("Waiting model evaluation is in process")
job.wait()

In the results from last step, click on the generated link to see your run in the Cloud Console.

In the UI, many of the pipeline DAG nodes will expand or collapse when you click on them. Here is a partially-expanded view of the DAG (click image to see larger version).
<img src="images/automl_tabular_regression_evaluation_pipeline.PNG" style="height:622px;width:726px"></img>

### Get the Model Evaluation Results

After the evalution pipeline is finished, run the below cell to print the evaluation metrics.

In [ ]:
model_evaluation = job.get_model_evaluation()

In [ ]:
# Iterate over the pipeline tasks
for (
    task
) in model_evaluation._backing_pipeline_job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the evaluation task
    if (
        ("model-evaluation" in task.task_name)
        and ("model-evaluation-import" not in task.task_name)
        and (
            task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
            or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
        )
    ):
        evaluation_metrics = task.outputs.get("evaluation_metrics").artifacts[
            0
        ]  # ['artifacts']
        evaluation_metrics_gcs_uri = evaluation_metrics.uri

print(evaluation_metrics)
print(evaluation_metrics_gcs_uri)

### Visualize the metrics


After the evalution pipeline is finished, run the below cell to visualize the evaluation metrics.

In [ ]:
metrics = []
values = []
for i in evaluation_metrics.metadata.items():
    if (
        i[0] == "meanAbsolutePercentageError"
    ):  # we are not considering MAPE as it is infinite. MAPE is infinite if groud truth is 0 as in our case Age is 0 for some instances.
        continue
    metrics.append(i[0])
    values.append(i[1])
plt.figure(figsize=(10, 5))
plt.bar(x=metrics, height=values)
plt.title("Evaluation Metrics")
plt.ylabel("Value")
plt.show()

### Get the Feature Attributions

Feature attributions indicate how much each feature in your model contributed to the predictions for each given instance.

Learn more about [Feature Attributions](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview#feature_attributions)

Run the below cell to get the feature attributions. 

In [ ]:
# Iterate over the pipeline tasks
for (
    task
) in model_evaluation._backing_pipeline_job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the feature-attribution task
    if (task.task_name == "feature-attribution") and (
        task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
        or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
    ):
        feat_attrs = task.outputs.get("feature_attributions").artifacts[0]
        feat_attrs_gcs_uri = feat_attrs.uri

print(feat_attrs)
print(feat_attrs_gcs_uri)

From the obtained Cloud Storage uri for the feature attributions, get the attribution values.

In [ ]:
# Load the results
attributions = !gsutil cat $feat_attrs_gcs_uri

# Print the results obtained
attributions = json.loads(attributions[0])
print(attributions)

### Visualize the Feature Attributions

Visualize the obtained attributions for each feature using a bar-chart.

In [ ]:
data = attributions["explanation"]["attributions"][0]["featureAttributions"]
features = []
attr_values = []
for key, value in data.items():
    features.append(key)
    attr_values.append(value)

plt.figure(figsize=(5, 3))
plt.bar(x=features, height=attr_values)
plt.title("Feature Attributions")
plt.xticks(rotation=90)
plt.ylabel("Attribution value")
plt.show()

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

Set `delete_bucket` to **True** to create the Cloud Storage bucket created in this notebook.

In [ ]:
import os

# Delete model resource
model.delete()

# Delete the dataset resource
dataset.delete()

# Delete the training job
train_job.delete()

# Delete the evaluation pipeline
job.delete()

# Delete Cloud Storage objects
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI